In [1]:
import torch

torch.manual_seed(456)

N, d = 20, 8

Q_mat = torch.rand((N, d))
K_mat = torch.rand((N, d))
V_mat = torch.rand((N, d))

In [2]:
Bi = 2
Bj = 2
O = torch.zeros((N, d))
S = torch.zeros((N, N))
for block_start_Bi in range(0, N, Bi):
    block_end_Bi = block_start_Bi + Bi
    Qi = Q_mat[block_start_Bi:block_end_Bi, :]  # shape Br x d
    acc = torch.zeros(Bi, d)
    for block_start_j in range(0, N, Bj):
        block_end_Bj = block_start_j + Bj
        Kj = K_mat[block_start_j:block_end_Bj, :]  # shape Br x d
        Sj = Qi @ Kj.T  # shape Br x Br
        Vj = V_mat[block_start_j:block_end_Bj, :]  # shape Br x d
        acc += Sj @ Vj

    O[block_start_Bi:block_end_Bi, :] += acc

assert block_end_Bi == block_end_Bj == N
assert torch.allclose(O, (Q_mat @ K_mat.T) @ V_mat)
